# section 1: derivatives

In [2]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [5]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  # return [0, 0, 0] # todo, return [df/da, df/db, df/dc]
    grad_a = -3.0 * (a ** 2) - 0.5 * (a**(-0.5))
    grad_b = 3.0 * cos(3*b) + 2.5 * (b ** 1.5)
    grad_c = c**(-2)
    return [grad_a, grad_b, grad_c]

# expected answer is the list of 
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [18]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

h = 1e-6

a = 2.0
b = 3.0
c = 4.0

grad_a = (f(a + h, b, c) - f(a, b, c))/h
grad_b = (f(a, b + h, c) - f(a, b, c))/h
grad_c = (f(a, b, c + h) - f(a, b, c))/h


# -----------
numerical_grad = [grad_a, grad_b, grad_c] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


### symmetric derivative

In [20]:
# there is an alternative formula that provides a much better numerical 
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

h = 1e-6

a = 2.0
b = 3.0
c = 4.0

grad_a = (f(a + h, b, c) - f(a - h, b, c)) / (2.0 * h)
grad_b = (f(a, b + h, c) - f(a, b - h, c)) / (2.0 * h)
grad_c = (f(a, b, c + h) - f(a, b, c - h)) / (2.0 * h)



# -----------
numerical_grad2 = [grad_a, grad_b, grad_c] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553391353245
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027401572
OK for dim 2: expected 0.0625, yours returns 0.06250000028629188


# section 2: support for softmax

In [68]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
        self.grad += other.data * out.grad
        other.grad += self.data * out.grad
    out._backward = _backward

    return out
  
  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out
  
  # ------
  # re-implement all the other functions needed for the exercises below
  def exp(self):
    out = Value(exp(self.data), (self,), 'exp')

    def _backward():
        self.grad += out.data * out.grad
    out._backward = _backward

    return out

  def log(self):
    out = Value(log(self.data), (self,), 'log')

    def _backward():
        self.grad += (1/self.data) * out.grad
    out._backward = _backward

    return out

  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int or float for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += (other) * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out

  def __radd__(self, other):
    return self + other

  def __truediv__(self, other):
    return self * other**-1

  def __rmul__(self, other):
    return self * other

  def __neg__(self):
      return self * -1

  # ------

  def backward(self): # exactly as in video  
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [89]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  # why does this work even though the c's are from different objects?
  # its because the logits are the same object even though logit.exp() is different so grads get deposited the same way
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)] # from 1 forward-pass, not 4 different ones
probs = softmax(logits)
print(probs)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


counts2 ids: [13344460480, 13344458848, 13344461440, 13344460048]
count ids: [13344463840, 13344463264, 13344470416, 13344458704]
[Value(data=0.04177257051535045), Value(data=0.839024507462532), Value(data=0.00565330266221633), Value(data=0.11354961935990122)]
2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


## in pytorch

In [186]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch
import torch.nn.functional as F

In [187]:
logit_tensor = torch.Tensor([0, 3, -2, 1]).double()
logit_tensor.requires_grad = True
logit_tensor

tensor([ 0.,  3., -2.,  1.], dtype=torch.float64, requires_grad=True)

In [188]:
loss = -F.log_softmax(logit_tensor, dim=-1)[3]
loss.backward()
loss.data.item()

2.1755153626167143

In [189]:
logit_tensor.grad

tensor([ 0.0418,  0.8390,  0.0057, -0.8865], dtype=torch.float64)

## tanh vs softmax

In [153]:
import numpy as np

input_vector = np.array([1, 2, 3])
tanh_output = np.tanh(input_vector)
print('tanh', tanh_output)

# softmax considers the entire vector
def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x)

softmax_output = softmax(input_vector)
print('softmax', softmax_output)

tanh [0.76159416 0.96402758 0.99505475]
softmax [0.09003057 0.24472847 0.66524096]


In [160]:
example = np.array([1, 2, 3, 4])
example / np.sum(example)

array([0.1, 0.2, 0.3, 0.4])